In [1]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate 
from langchain_community.vectorstores import Chroma
import pandas as pd

load document

In [2]:
df = pd.read_csv("realistic_restaurant_reviews.csv")
df


,Title,Date,Rating,Review
0,Best pizza in town,2024-03-15,5,The crust was perfectly crispy on the outside ...
1,Disappointed with service,2024-02-20,2,"While the pizza itself was decent, we waited o..."
2,Authentic Italian experience,2024-01-05,5,This place reminds me of the pizzerias in Napl...
3,Overpriced for what you get,2024-03-01,3,$24 for a medium pizza with just two toppings ...
4,Great gluten-free option,2024-02-15,4,"As someone with celiac disease, finding good g..."
...,...,...,...,...
118,Perfect for dietary restrictions,2024-02-09,5,"As someone with multiple food allergies, I app..."
119,Tasteless cheese substitute,2024-03-01,2,Their vegan cheese option was terrible - had n...
120,Masterclass in proper baking,2024-01-10,5,Watching the pizzaiolo work the oven is like w...
121,Tomato sauce tasted rancid,2024-02-16,1,The sauce had a distinctly off flavor that sug...


In [3]:
documents = [
    f"Title: {row['Title']}\nDate: {row['Date']}\nRating: {row['Rating']}\nReview: {row['Review']}"
    for _, row in df.iterrows()
]

Create embeddings and vectorstore

In [4]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
vectorstore = Chroma.from_texts(documents, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

Initialize LLM

In [5]:
llm = OllamaLLM(model="llama3.2", temperature=0.3)


Build the RetrievalQA Chain

In [6]:
from langchain.chains import RetrievalQA

# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retriever,
    chain_type = "stuff"

)

In [7]:
query = "What is the best pizza place mentioned in the reviews and best rating?"
answer = qa_chain.run(query)
print(answer)

C:\Users\vk133\AppData\Local\Temp\ipykernel_13280\1220017256.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


Based on the context provided, I can answer that there are multiple pizza places mentioned with high ratings (5-star reviews), but it's not clear which one is specifically referred to as "the best" since all three places receive 5-star reviews.


In [8]:
qa_chain.run("Summarize all negative reviews.")
qa_chain.run("Which review mentioned wood-fired Margherita pizza?")
qa_chain.run("What is the overall rating trend in February 2024?")


"Based on the provided reviews, I can see a general downward trend in ratings for February 2024. The first review of February (Title: Inconsistent quality) has a rating of 3, which is lower than the previous review from January (Title: Outstanding variety of options) with a rating of 5.\n\nThe second review of February (Title: All style no substance) also has a rating of 3, maintaining the same level as the first review. The third review of February (Title: Perfect crust-to-topping ratio) has a rating of 5, which is higher than the previous two reviews.\n\nHowever, I don't have enough information to determine if this trend continues throughout February or if it's just a snapshot of the month."